## Import statements

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

np.set_printoptions(precision=1)
np.set_printoptions(suppress=True)

random.seed(1)
torch.manual_seed(1)

import pickle
import time

## Dataset loading

**Dataset description**:

**Size**: 1,315,337

**Dataset Format**: 
1. home_lineup
2. away_lineup
3. box_stats
4. (num_possessions, score_update)

**Box Stat Format**: 10 x 13 array
1. PTS
2. TS%
3. FGA
4. AST
5. REB
6. STL
7. BLK
8. 2PA
9. 2PM
10. 3PA
11. 3PM
12. FTA
13. FTM

In [2]:
with open("processed_data/pickled_lineups1998-2018",'rb') as infile:
    lineups = pickle.load(infile)
    
with open("processed_data/lineup_metadata1998-2018",'rb') as infile:
    player_to_idx, idx_to_player, normal_name_to_idx, all_stats = pickle.load(infile)
    
print ("Dataset size: " + str(len(lineups)))

Dataset size: 1315337


In [3]:
# finding the statistics of the dataset
total_possessions = 0
total_points = 0
for l in lineups:
    pos, pts = l[3]
    total_possessions += pos
    total_points += pos * pts
avg_ppp = total_points / total_possessions
print ("Average points per possession: " + str(avg_ppp))

# filtering the dataset to only those with 5 players in both the offense and defense team
valid_indices = [i for i in range(len(lineups)) if len(lineups[i][0]) == len(lineups[i][1]) == 5]
filtered_dataset_size = len(valid_indices)
print ("Filtered dataset size: " + str(filtered_dataset_size))

Average points per possession: 0.9638286945483489
Filtered dataset size: 1313336


# RAPM model for comparison

In [4]:
class RAPM_Model(nn.Module):
    def __init__(self, n_players):
        super(RAPM_Model, self).__init__()
        self.n = n_players
        # embedding layer to transform players to embeddings
        self.o_layer = nn.Linear(n_players, 1)
        self.d_layer = nn.Linear(n_players, 1)
        
        
        
    def forward(self, inputs):
        off_one_hot = torch.sum(F.one_hot(inputs[0], self.n),dim=0).type(torch.FloatTensor).cuda()
        def_one_hot = torch.sum(F.one_hot(inputs[1], self.n),dim=0).type(torch.FloatTensor).cuda()
     
        return self.o_layer(off_one_hot) -  self.d_layer(def_one_hot)

## Model definition

In [6]:
class Embedding_Model(nn.Module):
    def __init__(self, n_players, d_model):
        super(Embedding_Model, self).__init__()
        
        self.d_model = d_model
        
        # embedding layer to transform players to embeddings
        self.embeds = nn.Embedding(n_players, d_model)
        
        # double linear layer for the offensive team
        self.offense_1 = nn.Linear(d_model, d_model)
        self.offense_2 = nn.Linear(d_model, d_model)
        #self.o_norm = nn.BatchNorm1d(d_model)
        
        # double linear layer for the defensive team
        self.defense_1 = nn.Linear(d_model, d_model)
        self.defense_2 = nn.Linear(d_model, d_model)
        #self.d_norm = nn.BatchNorm1d(d_model)
        self.to_o_stats_1 = nn.Linear(2*d_model,d_model) # there are 13 stats tracked
        self.to_d_stats_1 = nn.Linear(2*d_model,d_model)
        self.to_o_stats_2 = nn.Linear(d_model,13) 
        self.to_d_stats_2 = nn.Linear(d_model,13)
        
        # final linear layers for the box stats and the score change
        self.to_pm_1 = nn.Linear(d_model, d_model)
        self.to_pm_2 = nn.Linear(d_model,1)
        
        
    def forward(self, inputs):
        o = self.embeds(inputs[0])
        d = self.embeds(inputs[1])
        
        # calculating the team encoding
        team = torch.zeros(len(o[0]), self.d_model).cuda()
        
        for player in o:
            team += (self.offense_2(F.relu(self.offense_1(player))))
            
        for player in d:
            team += (self.defense_2(F.relu(self.defense_1(player))))

        # calculating the box stats
        boxstats = []
        
        for player in o:
            player = self.to_o_stats_1(torch.cat((team,player), dim = 1))
            boxstats.append(F.relu(self.to_o_stats_2(player)))
            
        for player in d:
            player = self.to_d_stats_1(torch.cat((team,player), dim = 1))
            boxstats.append(F.relu(self.to_d_stats_2(player)))
            
        boxstats = torch.stack(boxstats, dim = 1)
        
        # calculating the score change
        plusminus = F.relu(self.to_pm_1(team))
        plusminus = (self.to_pm_2(team)).reshape(-1)
        
        return boxstats, plusminus

## Training the model

**Computation configurations**

In [21]:
# checking if cuda is supported
use_cuda = torch.cuda.is_available()

# setting configurations depending on computing device
if use_cuda:
    computing_device = torch.device("cuda")
    num_workers = 1
    pin_memory = True
    print("Computing device: CUDA")
else:
    computing_device = torch.device("cpu")
    num_workers = 0
    pin_memory = False
    print("Computing device: CPU")


Computing device: CUDA



**RAPM baseline model**

In [22]:
rapm_model = RAPM_Model(len(idx_to_player)).to(computing_device)

**Model configurations**

In [55]:

# setting the model parameters
model_embedding_dimension = 256

embedding_model = Embedding_Model(len(idx_to_player), model_embedding_dimension).to(computing_device)
mme_model = Embedding_Model(len(idx_to_player), model_embedding_dimension).to(computing_device)

**Dataset configurations**

In [56]:
# setting the dataset configurations
percent_validation = 0.1
percent_testing = 0.2
batch_size = 512

# checking the size of the train/validation/test sets
num_validation = int(filtered_dataset_size * percent_validation)
num_testing = int(filtered_dataset_size * percent_testing)
num_training = filtered_dataset_size - (num_validation + num_testing)
print ()
print ("Training set size: " + str(num_training))
print ("Validation set size: " + str(num_validation))
print ("Testing set size: " + str(num_testing))

# shuffle the dataset
np.random.shuffle(valid_indices)

# split the dataset into the 3 sets
validation_indices = valid_indices[:num_validation]
testing_indices = valid_indices[num_validation : num_validation + num_testing]
training_indices = valid_indices[num_validation + num_testing:]

# create dataloaders to sample from the sets
training_sampler = SubsetRandomSampler(training_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
testing_sampler = SubsetRandomSampler(testing_indices)
                                      
training_loader = DataLoader(lineups, 
                             batch_size = batch_size, 
                             sampler = training_sampler, 
                             num_workers = num_workers, 
                             pin_memory = pin_memory)
                                      
validation_loader = DataLoader(lineups, 
                               batch_size = batch_size, 
                               sampler = validation_sampler, 
                               num_workers = num_workers, 
                               pin_memory = pin_memory)
                                      
testing_loader = DataLoader(lineups, 
                            batch_size = batch_size, 
                            sampler = testing_sampler, 
                            num_workers = num_workers, 
                            pin_memory = pin_memory)


Training set size: 919336
Validation set size: 131333
Testing set size: 262667


**Training configurations**

In [57]:
def weighted_MSE(output, target, num_pos):
    loss = torch.mean(torch.mul((output - target) ** 2, num_pos))
    return loss

In [58]:
# defining training parameters
epochs = 40
plusminus_priority = 30 # How much more we care about plusminus compared to box. 
embedding_optimizer = optim.Adam(embedding_model.parameters(), lr=0.002)
mme_optimizer = optim.Adam(mme_model.parameters(), lr=0.002)
rapm_optimizer = optim.Adam(rapm_model.parameters(), lr=0.1,weight_decay=0.001)

In [59]:
# creating a fake lineup for sanity checks
death_lineup = ['stephen curry', 'draymond green', 'klay thompson', 'andre iguodala', 'kevin durant']
death_lineup = [normal_name_to_idx[name] for name in death_lineup]
death_lineup = torch.LongTensor(death_lineup).reshape(-1, 1).to(computing_device)
print(death_lineup)

dead_lineup = ['ian clark', 'david west', 'shaun livingston', 'patrick mccaw', 'zaza pachulia']
dead_lineup = [normal_name_to_idx[name] for name in dead_lineup]
dead_lineup = torch.LongTensor(dead_lineup).reshape(-1, 1).to(computing_device)
print(dead_lineup)

rapm_model((death_lineup, dead_lineup))

tensor([[554],
        [551],
        [553],
        [ 38],
        [ 56]], device='cuda:0')
tensor([[825],
        [ 90],
        [454],
        [890],
        [ 14]], device='cuda:0')


tensor([[1.7261]], device='cuda:0', grad_fn=<SubBackward0>)

In [60]:
def validate(model, test_loader):
    with torch.no_grad():
        err = 0
        tot_pos = 0
        for i, (offense, defense, box, (num_pos, score_change)) in enumerate(test_loader):
            offense = torch.stack(offense).to(computing_device)
            defense = torch.stack(defense).to(computing_device)
            num_pos = num_pos.float().to(computing_device)
            score_change = score_change.float().to(computing_device)
            score_change = (score_change - avg_ppp) * 100   
            p_score_change = model((offense, defense))
            if type(p_score_change) is tuple:
                p_score_change = p_score_change[1]
            err += weighted_MSE(p_score_change, score_change, num_pos)
            tot_pos += torch.sum(num_pos)
    return err / tot_pos

**Training and Validation**

In [34]:
# training baseline model
training_losses = []
validation_losses = []

start_time = time.time()

for e in range(5):
    for i, (offense, defense, box, (num_pos, score_change)) in enumerate(training_loader):
        #offense, defense, box, n_pos, score_change = lineup
        
        offense = torch.stack(offense).to(computing_device)
        defense = torch.stack(defense).to(computing_device)
        num_pos = num_pos.float().to(computing_device)
        score_change = score_change.float().to(computing_device)
        
        # normalizing the score changes
        score_change = (score_change - avg_ppp) * 100

        rapm_optimizer.zero_grad()
        
        p_score_change = rapm_model((offense, defense))
        loss = weighted_MSE(p_score_change, score_change, num_pos)
        
        loss.backward()
        rapm_optimizer.step()
        
        training_losses.append(loss.item())
        
        if i % 500 == 0:
            print("Epoch: %d | Minibatch: %d | Loss: %.6f" % 
                  (e, i, loss.item()))
            
        if i % 1000 == 0:
            print()
            print("starters: ",rapm_model((death_lineup, dead_lineup))[0].item())
            print("bench: ",rapm_model((dead_lineup, death_lineup))[0].item())

Epoch: 0 | Minibatch: 0 | Loss: 12250.871094

starters:  -0.9105673432350159
bench:  -0.8839726448059082
Epoch: 0 | Minibatch: 500 | Loss: 13649.988281
Epoch: 0 | Minibatch: 1000 | Loss: 12470.527344

starters:  -0.5719029307365417
bench:  -0.8272386193275452
Epoch: 0 | Minibatch: 1500 | Loss: 13709.886719
Epoch: 1 | Minibatch: 0 | Loss: 12290.431641

starters:  -0.9515997171401978
bench:  -0.5513460040092468
Epoch: 1 | Minibatch: 500 | Loss: 12842.537109
Epoch: 1 | Minibatch: 1000 | Loss: 14492.257812

starters:  -0.8080108165740967
bench:  -0.3506311774253845
Epoch: 1 | Minibatch: 1500 | Loss: 13511.029297
Epoch: 2 | Minibatch: 0 | Loss: 11517.854492

starters:  1.43043851852417
bench:  1.075697898864746
Epoch: 2 | Minibatch: 500 | Loss: 11332.833008
Epoch: 2 | Minibatch: 1000 | Loss: 11262.287109

starters:  1.5173511505126953
bench:  1.970441222190857
Epoch: 2 | Minibatch: 1500 | Loss: 12743.937500
Epoch: 3 | Minibatch: 0 | Loss: 11127.666992

starters:  -0.12561702728271484
bench:

In [35]:
print(validate(rapm_model, testing_loader))
print(validate(rapm_model, testing_loader))

tensor(5.9166, device='cuda:0')
tensor(5.9157, device='cuda:0')


In [36]:
# training the embedding model
training_losses = []
validation_losses = []

start_time = time.time()

for e in range(epochs):
    #total_loss = 0
    #loss = torch.zeros(1).to(computing_device)
    #tot_pos = 0
    for i, (offense, defense, box, (num_pos, score_change)) in enumerate(training_loader):
        #offense, defense, box, n_pos, score_change = lineup
        
        offense = torch.stack(offense).to(computing_device)
        defense = torch.stack(defense).to(computing_device)
        box = box.float().to(computing_device)
        num_pos = num_pos.float().to(computing_device)
        score_change = score_change.float().to(computing_device)
        
        # normalizing the score changes
        score_change = (score_change - avg_ppp) * 100
        box = box * 100    

        embedding_optimizer.zero_grad()
        
        p_box, p_score_change = embedding_model((offense, defense))
        loss_box = 0 # no box stats in this one
        loss_pm = weighted_MSE(p_score_change, score_change, num_pos)
        loss = loss_box + loss_pm
        
        loss.backward()
        embedding_optimizer.step()
        
        training_losses.append(loss.item())
        
        if i % 500 == 0:
            print("Epoch: %d | Minibatch: %d | Loss: %.6f" % 
                  (e, i, loss.item()))
            
        if i % 1000 == 0:
            print()
            print("starters: ",embedding_model((death_lineup, dead_lineup))[1].item())
            print("bench: ",embedding_model((dead_lineup, death_lineup))[1].item())
            print(all_stats)
            print(embedding_model((death_lineup, dead_lineup))[0][0].cpu().data.numpy())

torch.save(embedding_model.state_dict(), "embedding_model.pt")

Epoch: 0 | Minibatch: 0 | Loss: 13194.766602

starters:  4.533792972564697
bench:  3.7042760848999023
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.5 0.2 0.  0.5 0.  0.3 0.2 0.4 0.  0.  0.  0.  0. ]
 [0.5 0.1 0.  0.3 0.1 0.1 0.  0.  0.1 0.  0.3 0.4 0.1]
 [0.4 0.  0.4 0.4 0.4 0.2 0.1 0.2 0.  0.  0.  0.5 0.2]
 [0.6 0.5 0.  0.1 0.2 0.3 0.  0.  0.  0.  0.1 0.4 0. ]
 [0.5 0.  0.  0.1 0.2 0.4 0.2 0.2 0.  0.  0.2 0.1 0. ]
 [0.3 0.  0.  0.  0.  0.1 0.  0.  0.  0.  0.3 0.3 0.3]
 [0.1 0.  0.  0.4 0.  0.7 0.  0.  0.  0.  0.  0.  0. ]
 [0.1 0.2 0.  0.  0.  0.3 0.  0.  0.  0.  0.  0.  0. ]
 [0.8 0.  0.  0.  0.  0.5 0.  0.  0.1 0.  0.  0.5 0. ]
 [0.2 0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.7 0.2 0.1]]
Epoch: 0 | Minibatch: 500 | Loss: 12093.669922
Epoch: 0 | Minibatch: 1000 | Loss: 12511.724609

starters:  10.02951431274414
bench:  -4.360506534576416
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.  2.1 3

Epoch: 5 | Minibatch: 500 | Loss: 11477.775391
Epoch: 5 | Minibatch: 1000 | Loss: 12401.035156

starters:  18.789901733398438
bench:  -10.280967712402344
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.4 0.  8.  0.  0.  2.9 1.  0.9 0.  0.7 0.7 4.5 0. ]
 [0.5 0.  8.5 0.  0.  2.6 0.8 0.2 0.5 0.5 1.4 5.  0. ]
 [0.4 0.  8.9 0.  0.  2.8 1.  0.6 0.3 0.1 0.7 4.9 0. ]
 [0.6 0.  8.1 0.  0.  2.8 0.8 0.2 0.2 0.2 1.2 4.9 0. ]
 [0.5 0.  8.5 0.  0.  2.9 1.  0.6 0.  0.5 1.3 4.5 0. ]
 [0.  0.  4.  0.4 1.1 0.  0.  0.  0.  2.7 0.  0.  1.2]
 [0.  0.  4.1 1.  1.2 0.  0.  0.  0.  2.5 0.  0.  0.7]
 [0.  0.  4.  0.4 1.  0.  0.  0.4 0.  2.3 0.  0.  0.8]
 [0.  0.  3.7 0.7 1.1 0.  0.  0.  0.  2.4 0.  0.  0.6]
 [0.  0.  3.9 0.7 0.8 0.  0.  0.  0.  2.7 0.  0.  1. ]]
Epoch: 5 | Minibatch: 1500 | Loss: 13014.815430
Epoch: 6 | Minibatch: 0 | Loss: 13123.653320

starters:  19.704954147338867
bench:  -7.123148441314697
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2

Epoch: 10 | Minibatch: 1500 | Loss: 12669.947266
Epoch: 11 | Minibatch: 0 | Loss: 11050.548828

starters:  23.416362762451172
bench:  -7.692562580108643
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.6 0.  4.9 0.  1.7 5.5 2.6 0.  0.  3.5 0.  2.4 0. ]
 [0.7 0.  5.3 0.  1.8 5.2 2.5 0.  0.  3.3 0.2 2.9 0. ]
 [0.6 0.  5.8 0.  2.1 5.4 2.7 0.  0.  2.9 0.  2.8 0. ]
 [0.8 0.  5.  0.  2.  5.3 2.5 0.  0.  3.  0.  2.8 0. ]
 [0.7 0.  5.4 0.  2.  5.5 2.7 0.  0.  3.3 0.1 2.4 0. ]
 [0.  0.  3.6 0.  0.  0.  0.  0.  0.  2.  0.  0.1 0. ]
 [0.  0.  3.7 0.6 0.  0.  0.  0.  0.  1.7 0.  0.  0. ]
 [0.  0.  3.6 0.  0.  0.  0.  0.  0.  1.6 0.  0.  0. ]
 [0.  0.  3.3 0.2 0.  0.  0.  0.  0.  1.6 0.  0.3 0. ]
 [0.  0.  3.5 0.2 0.  0.  0.  0.  0.  1.9 0.  0.1 0. ]]
Epoch: 11 | Minibatch: 500 | Loss: 11003.484375
Epoch: 11 | Minibatch: 1000 | Loss: 12877.952148

starters:  19.327192306518555
bench:  -8.726744651794434
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK',

Epoch: 16 | Minibatch: 500 | Loss: 12394.569336
Epoch: 16 | Minibatch: 1000 | Loss: 12453.722656

starters:  20.094078063964844
bench:  -9.13140869140625
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.8 0.  2.4 0.  0.  1.4 1.7 1.1 0.  4.4 0.  1.  0. ]
 [0.9 0.  2.8 0.  0.  1.1 1.7 0.5 0.  4.3 0.4 1.5 0. ]
 [0.8 0.  3.2 0.  0.  1.3 1.8 0.9 0.  3.8 0.  1.4 0. ]
 [1.  0.  2.5 0.  0.  1.2 1.7 0.5 0.  4.  0.2 1.4 0. ]
 [0.9 0.  2.9 0.  0.  1.4 1.9 0.9 0.  4.3 0.3 1.  0. ]
 [0.  0.  0.  0.7 0.  0.  0.  0.  0.  1.2 0.  0.  2.1]
 [0.  0.  0.  1.3 0.  0.  0.  0.  0.  1.  0.  0.  1.6]
 [0.  0.  0.  0.6 0.  0.  0.  0.  0.  0.8 0.  0.  1.8]
 [0.  0.  0.  0.9 0.  0.  0.  0.  0.1 0.9 0.  0.  1.5]
 [0.  0.  0.  0.8 0.  0.  0.  0.  0.  1.2 0.  0.  1.9]]
Epoch: 16 | Minibatch: 1500 | Loss: 11954.470703
Epoch: 17 | Minibatch: 0 | Loss: 13120.334961

starters:  23.837623596191406
bench:  -10.62806510925293
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', 

Epoch: 21 | Minibatch: 1500 | Loss: 12382.344727
Epoch: 22 | Minibatch: 0 | Loss: 13092.853516

starters:  22.809144973754883
bench:  -9.476619720458984
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.6 0.  3.3 0.9 0.  2.1 1.6 1.5 0.  4.6 0.  0.5 0. ]
 [0.7 0.  3.6 0.7 0.  1.8 1.5 1.  0.  4.5 0.5 1.1 0. ]
 [0.6 0.  4.  0.8 0.  2.  1.7 1.3 0.  4.1 0.  0.9 0. ]
 [0.8 0.  3.3 0.7 0.  1.9 1.6 0.9 0.  4.2 0.3 0.9 0. ]
 [0.8 0.  3.7 0.6 0.  2.1 1.8 1.3 0.  4.5 0.4 0.5 0. ]
 [0.  0.  1.3 0.  0.  0.  0.  0.  0.  1.6 0.  0.  1.1]
 [0.  0.  1.3 0.3 0.  0.  0.  0.  0.  1.3 0.  0.  0.7]
 [0.  0.  1.3 0.  0.  0.  0.  0.  0.  1.2 0.  0.  0.8]
 [0.  0.  1.  0.  0.  0.  0.  0.  0.  1.3 0.  0.  0.6]
 [0.  0.  1.2 0.  0.  0.  0.  0.  0.  1.6 0.  0.  1. ]]
Epoch: 22 | Minibatch: 500 | Loss: 11872.000000
Epoch: 22 | Minibatch: 1000 | Loss: 12622.897461

starters:  26.325439453125
bench:  -8.853597640991211
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2

Epoch: 27 | Minibatch: 500 | Loss: 11968.736328
Epoch: 27 | Minibatch: 1000 | Loss: 10114.283203

starters:  19.994068145751953
bench:  -9.397347450256348
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.1 0.  1.9 0.  0.  2.2 1.2 0.9 0.  4.3 0.  1.4 0. ]
 [0.2 0.  2.2 0.  0.  1.8 1.1 0.4 0.  4.2 0.  1.9 0. ]
 [0.  0.  2.7 0.  0.  2.  1.3 0.8 0.  3.8 0.  1.7 0. ]
 [0.2 0.  1.9 0.  0.  1.9 1.2 0.4 0.  4.  0.  1.7 0. ]
 [0.2 0.  2.4 0.  0.  2.1 1.4 0.7 0.  4.2 0.  1.4 0. ]
 [0.  0.  1.7 0.  0.  0.  0.  0.  0.5 1.8 0.  0.  1.9]
 [0.  0.  1.6 0.  0.  0.  0.  0.  0.1 1.5 0.  0.  1.4]
 [0.  0.  1.6 0.  0.  0.  0.  0.  0.5 1.4 0.  0.  1.6]
 [0.  0.  1.3 0.  0.  0.  0.  0.  0.7 1.5 0.  0.  1.3]
 [0.  0.  1.6 0.  0.  0.  0.  0.  0.5 1.7 0.  0.  1.7]]
Epoch: 27 | Minibatch: 1500 | Loss: 12268.637695
Epoch: 28 | Minibatch: 0 | Loss: 12199.864258

starters:  16.59529685974121
bench:  -9.764455795288086
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', 

Epoch: 32 | Minibatch: 1500 | Loss: 11962.806641
Epoch: 33 | Minibatch: 0 | Loss: 12687.742188

starters:  19.375755310058594
bench:  -9.40811824798584
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[0.3 0.  3.4 0.  0.  3.5 1.3 1.8 0.  3.4 0.  1.7 0. ]
 [0.4 0.  3.7 0.  0.  3.1 1.3 1.4 0.6 3.4 0.2 2.2 0. ]
 [0.2 0.  4.2 0.  0.  3.3 1.4 1.7 0.3 2.9 0.  2.  0. ]
 [0.5 0.  3.5 0.  0.  3.1 1.3 1.3 0.3 3.1 0.  2.1 0. ]
 [0.3 0.  3.9 0.  0.  3.5 1.5 1.7 0.  3.3 0.1 1.7 0. ]
 [0.  0.  2.3 0.  0.5 0.  0.  0.  0.5 1.8 0.  0.  2.4]
 [0.  0.  2.2 0.4 0.7 0.  0.  0.  0.  1.5 0.  0.  1.9]
 [0.  0.  2.2 0.  0.4 0.  0.  0.  0.3 1.5 0.  0.  2.1]
 [0.  0.  1.8 0.  0.5 0.  0.  0.  0.5 1.5 0.  0.  1.7]
 [0.  0.  2.1 0.  0.3 0.  0.  0.  0.4 1.7 0.  0.  2.2]]
Epoch: 33 | Minibatch: 500 | Loss: 11230.750000
Epoch: 33 | Minibatch: 1000 | Loss: 12436.014648

starters:  17.435991287231445
bench:  -9.997008323669434
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', 

Epoch: 38 | Minibatch: 500 | Loss: 11344.851562
Epoch: 38 | Minibatch: 1000 | Loss: 12344.623047

starters:  15.302700996398926
bench:  -8.494345664978027
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[1.4 0.  2.8 0.8 0.  1.6 1.3 0.5 0.  2.5 0.1 1.2 0. ]
 [1.5 0.  3.2 0.6 0.  1.2 1.3 0.1 0.  2.4 0.8 1.8 0. ]
 [1.4 0.  3.6 0.7 0.2 1.5 1.4 0.4 0.  2.  0.2 1.5 0. ]
 [1.6 0.  2.9 0.6 0.  1.2 1.3 0.  0.  2.2 0.5 1.6 0. ]
 [1.5 0.  3.3 0.5 0.  1.5 1.5 0.4 0.  2.4 0.6 1.2 0. ]
 [0.  0.  2.  0.  0.5 0.  0.  0.  0.  1.4 0.  0.  1.4]
 [0.  0.  2.  0.5 0.6 0.  0.  0.  0.  1.1 0.  0.  1. ]
 [0.  0.  2.  0.  0.4 0.  0.  0.  0.  1.  0.  0.  1.1]
 [0.  0.  1.6 0.1 0.4 0.  0.  0.  0.  1.1 0.  0.  0.8]
 [0.  0.  1.9 0.1 0.3 0.  0.  0.  0.  1.3 0.  0.  1.2]]
Epoch: 38 | Minibatch: 1500 | Loss: 11677.924805
Epoch: 39 | Minibatch: 0 | Loss: 13913.035156

starters:  21.534664154052734
bench:  -9.839545249938965
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK',

In [37]:
embedding_model = Embedding_Model(len(idx_to_player), model_embedding_dimension).to(computing_device)
embedding_model.load_state_dict(torch.load("embedding_model.pt"))
embedding_model.eval()

Embedding_Model(
  (embeds): Embedding(1949, 256)
  (offense_1): Linear(in_features=256, out_features=256, bias=True)
  (offense_2): Linear(in_features=256, out_features=256, bias=True)
  (defense_1): Linear(in_features=256, out_features=256, bias=True)
  (defense_2): Linear(in_features=256, out_features=256, bias=True)
  (to_o_stats_1): Linear(in_features=512, out_features=256, bias=True)
  (to_d_stats_1): Linear(in_features=512, out_features=256, bias=True)
  (to_o_stats_2): Linear(in_features=256, out_features=13, bias=True)
  (to_d_stats_2): Linear(in_features=256, out_features=13, bias=True)
  (to_pm_1): Linear(in_features=256, out_features=256, bias=True)
  (to_pm_2): Linear(in_features=256, out_features=1, bias=True)
)

In [38]:
print(validate(embedding_model, training_loader))
print(validate(embedding_model, testing_loader))

tensor(5.8134, device='cuda:0')
tensor(5.8656, device='cuda:0')


In [61]:
# training the multimodal embedding model
training_losses = []
validation_losses = []

start_time = time.time()

for e in range(epochs):
    #total_loss = 0
    #loss = torch.zeros(1).to(computing_device)
    #tot_pos = 0
    for i, (offense, defense, box, (num_pos, score_change)) in enumerate(training_loader):
        #offense, defense, box, n_pos, score_change = lineup
        
        offense = torch.stack(offense).to(computing_device)
        defense = torch.stack(defense).to(computing_device)
        box = box.float().to(computing_device)
        num_pos = num_pos.float().to(computing_device)
        score_change = score_change.float().to(computing_device)
        
        # normalizing the score changes
        score_change = (score_change - avg_ppp) * 100
        box = box * 100    

        mme_optimizer.zero_grad()
        
        p_box, p_score_change = mme_model((offense, defense))
        loss_box = torch.sum(weighted_MSE(p_box, box, num_pos.reshape(-1, 1, 1))) / plusminus_priority
        loss_pm = weighted_MSE(p_score_change, score_change, num_pos)
        loss = loss_box + loss_pm
        
        loss.backward()
        mme_optimizer.step()
        
        training_losses.append(loss.item())
        
        if i % 500 == 0:
            print("Epoch: %d | Minibatch: %d | Loss: %.6f" % 
                  (e, i, loss.item()))
            
        if i % 1000 == 0:
            print()
            print("starters: ",mme_model((death_lineup, dead_lineup))[1].item())
            print("bench: ",mme_model((dead_lineup, death_lineup))[1].item())
            print(all_stats)
            print(mme_model((death_lineup, dead_lineup))[0][0].cpu().data.numpy())
            
torch.save(mme_model.state_dict(), "mme_model.pt")

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f8ca40635c0>>
Traceback (most recent call last):
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aidan/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch: 0 | Minibatch: 0 | Loss: 12163.660156

starters:  -8.23121452331543
bench:  -8.329282760620117
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[1.4 2.9 0.9 0.  1.5 0.  0.2 0.  1.2 0.  0.  0.  1.1]
 [1.1 3.1 1.4 0.  1.3 0.1 0.  0.2 0.9 0.  0.4 0.  0.1]
 [1.2 3.4 1.2 0.4 1.6 0.  0.  0.  1.3 0.  0.2 0.  0.9]
 [1.3 3.1 1.1 0.1 1.8 0.  0.  0.  0.6 0.  0.  0.  0.9]
 [1.3 3.1 1.3 0.1 1.9 0.  0.2 0.1 1.1 0.  0.1 0.  0.8]
 [1.1 3.1 0.  0.  1.  0.4 0.8 0.  0.  0.2 0.  0.5 0.6]
 [1.  3.3 0.  0.  0.5 0.1 0.9 0.  0.  0.1 0.2 0.2 0.4]
 [0.9 4.  0.  0.  1.  0.4 1.  0.  0.  0.  0.1 0.8 0.7]
 [1.7 3.3 0.  0.  0.5 0.  0.7 0.2 0.  0.  0.  0.8 0.6]
 [0.5 4.3 0.  0.  1.2 0.2 0.9 0.3 0.  0.2 0.  1.  0.6]]
Epoch: 0 | Minibatch: 500 | Loss: 12474.427734
Epoch: 0 | Minibatch: 1000 | Loss: 12830.072266

starters:  12.425263404846191
bench:  -4.6542510986328125
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[27.8 48

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f8ca40635c0>>
Traceback (most recent call last):
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aidan/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch: 1 | Minibatch: 0 | Loss: 12531.486328

starters:  15.709976196289062
bench:  -7.796985626220703
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[21.7 36.9 17.7  8.7  0.   0.   0.   8.6  5.8  7.3  2.   3.3  1.6]
 [15.9 32.9 12.7  6.3  1.4  0.   0.   8.7  5.9  2.   0.   3.   1.2]
 [19.1 35.9 15.5  3.8  1.4  0.   0.   9.5  6.4  4.4  0.8  3.   1.1]
 [15.5 34.9 12.6  3.9  2.1  0.   0.   9.4  6.3  1.2  0.   3.1  1.1]
 [23.9 41.4 18.1  3.5  0.5  0.   0.  10.1  7.4  6.4  1.3  4.4  2.3]
 [ 2.2 11.7  0.   0.   2.4  0.6  0.   0.   0.   0.   0.   2.2  2.2]
 [ 2.3 13.5  0.   0.   5.7  0.2  0.   0.   0.   0.   0.   2.4  2.3]
 [ 1.8  9.3  0.   0.   4.4  0.7  0.   0.   0.   0.   0.   2.   1.8]
 [ 2.1  9.5  0.   0.   3.5  0.8  0.   0.   0.   0.   0.   2.2  2. ]
 [ 2.6 13.5  0.   0.   6.2  0.5  0.   0.   0.   0.   0.   2.7  2.5]]
Epoch: 1 | Minibatch: 500 | Loss: 13056.234375
Epoch: 1 | Minibatch: 1000 | Loss: 12165.841797

starters:  16.44994163513183

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f8ca40635c0>>
Traceback (most recent call last):
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aidan/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


...

Epoch: 37 | Minibatch: 0 | Loss: 11972.252930

starters:  17.829402923583984
bench:  -9.903132438659668
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[27.  51.8 21.1  9.   0.   0.   0.  11.5  6.4  9.7  4.   3.4  2.5]
 [15.  38.2 10.8  6.6  2.1  0.   0.   7.   4.7  3.4  1.5  2.4  1.2]
 [23.3 45.7 18.3  4.9  1.   0.   0.  10.9  6.3  7.8  3.5  2.   1.6]
 [14.5 38.1 10.3  4.5  1.7  0.   0.   6.6  4.3  4.   1.8  2.5  1.1]
 [29.2 55.  21.5  3.8  1.1  0.   0.  14.8  8.8  6.7  3.1  4.8  3.2]
 [ 0.7 15.1  0.   0.   4.   1.7  0.   0.   0.   0.   0.   0.7  0.6]
 [ 2.2 27.8  0.   0.   8.9  1.3  1.6  0.   0.   0.   0.   2.2  2.1]
 [ 0.6 16.6  0.   0.   6.8  1.4  0.   0.   0.   0.   0.   0.6  0.6]
 [ 0.3 11.7  0.   0.   4.7  2.   0.   0.   0.   0.   0.   0.2  0.2]
 [ 1.7 24.5  0.   0.  10.1  1.3  1.3  0.   0.   0.   0.   1.7  1.6]]
Epoch: 37 | Minibatch: 500 | Loss: 12726.173828
Epoch: 37 | Minibatch: 1000 | Loss: 13045.071289

starters:  22.59093284606

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f8ca40635c0>>
Traceback (most recent call last):
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aidan/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch: 38 | Minibatch: 0 | Loss: 13117.813477

starters:  22.45675277709961
bench:  -11.343172073364258
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[26.5 46.5 20.7  7.8  0.   0.   0.  11.2  6.   8.9  0.   3.2  0. ]
 [14.3 33.3 10.7  5.5  2.2  0.   0.   7.4  4.3  2.5  0.   1.6  0. ]
 [21.8 40.4 17.7  3.1  1.6  0.   0.  10.2  5.4  7.1  0.   1.7  0. ]
 [14.  34.6 10.5  3.6  1.6  0.   0.   6.9  4.1  3.4  0.   1.8  0. ]
 [29.9 52.8 21.9  3.   1.6  0.   0.  15.1  8.8  6.2  0.   4.8  0. ]
 [ 0.   9.7  0.   0.   3.1  1.4  0.   0.   0.   0.   0.   0.   0. ]
 [ 1.5 22.3  0.   0.   8.7  0.8  1.7  0.   0.   0.   0.   1.5  1.5]
 [ 0.4 11.5  0.   0.   6.4  1.5  0.   0.   0.   0.   0.   0.4  0.4]
 [ 0.   8.   0.   0.   4.6  1.5  0.   0.   0.   0.   0.   0.   0. ]
 [ 1.6 18.6  0.   0.   9.7  1.3  0.7  0.   0.   0.   0.   1.7  1.6]]
Epoch: 38 | Minibatch: 500 | Loss: 12637.985352
Epoch: 38 | Minibatch: 1000 | Loss: 12385.098633

starters:  20.90219116210

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f8ca40635c0>>
Traceback (most recent call last):
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aidan/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch: 39 | Minibatch: 0 | Loss: 13087.982422

starters:  17.922439575195312
bench:  -10.043062210083008
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[26.5 51.6 20.2  8.8  0.   0.   0.  11.   6.2  9.2  0.   2.6  0. ]
 [14.3 37.9 10.2  6.8  2.1  0.   0.   6.9  4.4  2.5  0.   2.   0. ]
 [22.3 45.2 17.8  4.5  1.   0.   0.  10.9  6.1  7.1  0.   1.7  0. ]
 [12.9 37.5  9.   4.4  1.7  0.   0.   6.   3.9  2.7  0.   2.   0. ]
 [28.  55.2 20.   3.7  1.2  0.   0.  14.1  8.2  5.8  0.   5.2  0. ]
 [ 0.  15.5  0.   0.   2.3  0.8  0.   0.   0.   0.   0.   0.   0. ]
 [ 1.5 30.1  0.   0.   8.7  1.   1.6  0.   0.   0.   0.   1.5  1.5]
 [ 0.1 15.9  0.   0.   6.1  1.3  0.   0.   0.   0.   0.   0.2  0.1]
 [ 0.3 14.   0.   0.   4.4  1.1  0.   0.   0.   0.   0.   0.3  0.3]
 [ 1.5 24.7  0.   0.   9.2  1.1  0.6  0.   0.   0.   0.   1.5  1.5]]
Epoch: 39 | Minibatch: 500 | Loss: 12470.429688
Epoch: 39 | Minibatch: 1000 | Loss: 10834.631836

starters:  23.0637969970

In [62]:
mme_model = Embedding_Model(len(idx_to_player), model_embedding_dimension).to(computing_device)
mme_model.load_state_dict(torch.load("mme_model.pt"))
mme_model.eval()

Embedding_Model(
  (embeds): Embedding(1949, 256)
  (offense_1): Linear(in_features=256, out_features=256, bias=True)
  (offense_2): Linear(in_features=256, out_features=256, bias=True)
  (defense_1): Linear(in_features=256, out_features=256, bias=True)
  (defense_2): Linear(in_features=256, out_features=256, bias=True)
  (to_o_stats_1): Linear(in_features=512, out_features=256, bias=True)
  (to_d_stats_1): Linear(in_features=512, out_features=256, bias=True)
  (to_o_stats_2): Linear(in_features=256, out_features=13, bias=True)
  (to_d_stats_2): Linear(in_features=256, out_features=13, bias=True)
  (to_pm_1): Linear(in_features=256, out_features=256, bias=True)
  (to_pm_2): Linear(in_features=256, out_features=1, bias=True)
)

In [63]:
print(validate(mme_model, training_loader))
print(validate(mme_model, testing_loader))

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f8ca40635c0>>
Traceback (most recent call last):
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aidan/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


tensor(5.8058, device='cuda:0')


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f8ca40635c0>>
Traceback (most recent call last):
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aidan/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aidan/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


tensor(5.8612, device='cuda:0')


In [42]:
i = 0
avg = 0
epoch_losses = []
l = len(training_losses)
for t in training_losses:
    avg += t
    i += 1
    if i > (l / 50):
        epoch_losses.append(avg / (l / 50))
        avg = 0
        i = 0
print(epoch_losses)

[12384.840010407243, 12365.005903690058, 12333.277270480627, 12316.026334698854, 12324.645636543664, 12304.162571828456, 12318.070926114022, 12304.602547626453, 12307.784852705448, 12310.29903556297, 12308.964082509745, 12289.62933172893, 12305.50656053165, 12292.056222078796, 12303.51743226593, 12295.520532688744, 12277.3109072407, 12303.373322688658, 12299.670461811838, 12299.102065462312, 12289.675518622113, 12275.737615980392, 12302.501865718306, 12289.451828852529, 12296.237591783825, 12283.962837338182, 12284.471848111907, 12301.006979485663, 12293.093887023246, 12280.914323632596, 12288.629783851267, 12287.285284029615, 12280.797079719256, 12290.914469763537, 12285.18497458545, 12286.857070752932, 12276.438641452578, 12279.07496802791, 12284.964908863229, 12296.561705455482, 12276.312336061474, 12291.953325505247, 12279.357597504002, 12282.423901802182, 12286.674265702211, 12279.21553851767, 12284.0893731404, 12295.405741056515, 12271.655134783025]


## Scratch work* 0.8

In [64]:
print("starters: ",mme_model((death_lineup, dead_lineup))[1].item())
print("bench: ",mme_model((dead_lineup, death_lineup))[1].item())
print(all_stats)
print(mme_model((death_lineup, dead_lineup))[0][0].cpu().data.numpy())

starters:  18.811973571777344
bench:  -9.257301330566406
['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
[[25.5 47.7 20.4  8.8  0.   0.   0.  11.   6.2  9.6  0.   3.4  2.6]
 [14.1 36.4 10.5  6.7  2.5  0.   0.   6.9  4.3  3.4  0.   2.5  1.4]
 [22.6 44.4 18.7  4.5  1.2  0.   0.  11.4  6.4  7.6  0.   2.   1.3]
 [13.7 36.6 10.2  4.9  1.5  0.   0.   6.6  4.4  3.9  0.   2.5  1.5]
 [28.3 52.4 21.1  3.4  1.6  0.   0.  14.5  8.8  7.   0.   5.4  3.8]
 [ 0.5 13.5  0.   0.   2.6  1.7  0.   0.   0.   0.   0.   0.5  0.5]
 [ 2.2 27.2  0.   0.   8.   1.   0.9  0.   0.   0.   0.   2.2  2.2]
 [ 0.7 14.1  0.   0.   5.5  1.6  0.   0.   0.   0.   0.   0.7  0.7]
 [ 0.6 12.5  0.   0.   4.8  2.   0.   0.   0.   0.   0.   0.6  0.6]
 [ 1.8 23.6  0.   0.   9.4  1.3  0.   0.   0.   0.   0.   1.8  1.8]]


In [82]:
import pandas as pd

In [119]:
death_lineup_names = ['stephen curry', 'draymond green', 'klay thompson', 'shaun livingston', 'andre iguodala']
death_lineup = [normal_name_to_idx[name] for name in death_lineup_names]
death_lineup = torch.LongTensor(death_lineup).reshape(-1, 1).to(computing_device)

dead_lineup_names = ['tim duncan', 'rudy gobert', 'draymond green', 'kevin garnett', 'robert covington']
dead_lineup = [normal_name_to_idx[name] for name in dead_lineup_names]
dead_lineup = torch.LongTensor(dead_lineup).reshape(-1, 1).to(computing_device)

print(all_stats)
death_stats = mme_model((death_lineup, dead_lineup))[0][0].cpu().data.numpy()[0:5] + \
     mme_model((dead_lineup, death_lineup))[0][0].cpu().data.numpy()[5:10]

print()

dead_stats = mme_model((death_lineup, dead_lineup))[0][0].cpu().data.numpy()[5:10] + \
     mme_model((dead_lineup, death_lineup))[0][0].cpu().data.numpy()[0:5]
    
stat_dict = {}
for i in range(5):
    stat_dict[death_lineup_names[i]] = death_stats[i]
    stat_dict[dead_lineup_names[i]] = dead_stats[i]
print()
t1 = mme_model((death_lineup, dead_lineup))[1].cpu().data.numpy()

t2 = mme_model((dead_lineup, death_lineup))[1].cpu().data.numpy()
print("Team 1 per 100 pos score differential: ", t1 - t2)
df = pd.DataFrame(stat_dict,index=all_stats)
df = df[death_lineup_names+dead_lineup_names]
df

['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']


Team 1 per 100 pos score differential:  [-9.5]


,stephen curry,draymond green,klay thompson,shaun livingston,andre iguodala,tim duncan,rudy gobert,draymond green,kevin garnett,robert covington
PTS,23.478975,14.554618,19.580559,7.212969,10.598179,24.129911,14.667349,14.554618,22.181854,13.294597
TS%,61.239735,54.635635,54.027382,34.526985,45.971809,74.148315,56.431519,54.635635,71.391266,54.068649
FGA,19.881266,12.618546,18.190279,8.345600,9.734591,19.884829,11.581074,12.618546,19.114977,11.119402
AST,7.493001,7.254947,3.238947,3.126966,3.650452,4.247034,1.649670,7.254947,4.795050,3.464233
REB,4.300941,10.775525,5.020619,8.165689,5.593816,16.054157,16.036787,10.775525,15.523176,10.227028
STL,2.272679,2.127357,1.305262,1.540582,1.723137,0.812743,1.348946,2.127357,1.454178,2.115356
BLK,0.000000,0.818042,0.000000,0.000000,0.000000,3.074854,2.542014,0.818042,1.475598,0.727681
2PA,11.475656,9.596780,11.844425,8.000201,7.068843,19.246313,10.305170,9.596780,18.215242,4.264428
2PM,4.634516,4.382730,4.848978,3.141707,2.842898,9.694060,5.543548,4.382730,8.797487,1.780257
3PA,8.418732,2.584611,6.376534,0.000000,2.718288,0.000000,1.099869,2.584611,0.099301,7.501280


In [ ]:
def random_order(dset):
    order = [i for i in range(len(dset))]
    random.shuffle(order)
    o_b, d_b, b_b, n_b, s_b = [], [], [], [], []
    samples = 0
    for i, o in enumerate(order):
        offense, defense, box, (n_pos, score_change) = dset[o]
        if len(offense) == 5 and len(defense) == 5:
            yield offense, defense, box, n_pos, score_change

In [ ]:
players = torch.LongTensor([1,2,3,4,5])

In [ ]:
e = nn.Embedding(20,5)
embeds = e(players)
for player in embeds:
    print(player)
    
torch.cat((embeds[1], embeds[2]), 0)
torch.stack((embeds[1], embeds[2]))

In [ ]:
model.offensive_embeddings(torch.LongTensor([normal_name_to_idx['stephen curry']]).cuda())[0]

In [ ]:
print(losses)

In [ ]:
print([l/100000000 for l in losses])

In [ ]:
torch.FloatTensor(box)